In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
batch_size = 25
img_height, img_width = 256, 256
input_shape = (img_height, img_width, 3)


def create_data_sparse():
    # data_gen = ImageDataGenerator(horizontal_flip=True, rotation_range=20, width_shift_range=0.2,
    #                               height_shift_range=0.2, zoom_range=0.2)
    data_gen = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

    train_images = data_gen.flow_from_directory('Dataset/PandasBears/Train', 
                                                target_size=(img_height, img_width),
                                                class_mode='binary', 
                                                batch_size=500, 
                                                subset='training', 
                                                # color_mode='rgb'
                                                )
    
    
    test_images = data_gen.flow_from_directory('Dataset/PandasBears/Test', 
                                               target_size=(img_height, img_width),
                                               class_mode='binary', 
                                               batch_size=200, 
                                               shuffle=False, 
                                            #    color_mode='rgb'
                                               )

    return train_images, test_images

train_images, test_images = create_data_sparse()

labels = ['bear', 'panda']
# print(train_images)


Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [27]:
print(len(train_images[0][0]))

400


In [35]:
X_train = train_images[0][0]
y_train = train_images[0][1]

X_test = test_images[0][0]
y_test = test_images[0][1]
# print(X_train)

In [ ]:


plt.figure(figsize = (20,20))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.subplots_adjust(hspace=0.3,wspace=0.3)
    plt.imshow(X_train[i])
    plt.title(f"Class: {labels[int(y_train[i])]}")
    plt.axis('off')


In [ ]:
samples = next(train_images)
print(samples[0][0].shape);

In [96]:
import numpy as np

class Pooling:
    def __init__(self,  filter_size, stride, mode):
        self.filter_size = filter_size
        self.stride = stride
        self.mode = mode

    def forward(self, input):
        input_size = input.shape
        output_size = (((input_size[0] - self.filter_size[0] ) // self.stride ) + 1, ((input_size[1] - self.filter_size[1]) // self.stride ) + 1)
        output = np.zeros((output_size[0], output_size[1],input_size[2]))
        for i in range(0,input_size[0] - self.filter_size[0] +1, self.stride):
            for j in range(0, input_size[1] - self.filter_size[1] +1 ,self.stride):
                for n in range(input_size[2]):
                    input_patch = input[i:i+self.filter_size[0], j:j+self.filter_size[1], n]
                    if self.mode == "max":
                        # Max pooling
                        output[i//self.stride][j//self.stride][n] = np.max(input_patch)
                    elif self.mode == "average":
                        # Average pooling
                        output[i//self.stride][j//self.stride][n] = np.mean(input_patch)
        return output
    
    def getModel(self):
        return ''


In [97]:
import numpy as np

class Convolution:
    
    # TODO: diferent width / height
    def __init__(self, input_size, padding_size, filter_size, num_filters, stride, bias):
        self.input_size = input_size
        self.padding_size = padding_size
        self.filter_size = filter_size

        self.num_filters = num_filters
        self.stride = stride
        # self.bias = bias

        self.output_size = (((input_size[0] - filter_size[0] + 2 * padding_size) // stride ) + 1, ((input_size[1] - filter_size[1] + 2 * padding_size) // stride ) + 1)
        # init random filter
        self.filter = [np.random.randn(self.filter_size[0], self.filter_size[1], input_size[2]) for _ in range(self.num_filters)]
        self.bias = np.zeros((self.output_size[0], self.output_size[1], self.num_filters))

    # include convolution and detector
    def forward(self, input):


        # Menambahkan padding jika diperlukan
        if self.padding_size > 0:
            padded_input = np.pad(input, ((self.padding_size, self.padding_size), (self.padding_size, self.padding_size), (0, 0)), mode='constant')
        else:
            padded_input = input

        # init input
        output = np.zeros((self.output_size[0], self.output_size[1], self.num_filters))

        for i in range(0,self.input_size[0] - self.filter_size[0] + (2 * self.padding_size) + 1, self.stride):
            for j in range(0, self.input_size[1] - self.filter_size[1] + (2 * self.padding_size) + 1,self.stride):
                for n in range(self.num_filters):
                    # Mengambil bagian input yang sesuai dengan ukuran filter
                    input_patch = padded_input[i:i+self.filter_size[0], j:j+self.filter_size[1]]
                    # Melakukan operasi konvolusi
                    output[i//self.stride][j//self.stride][n] = np.maximum(0,np.sum(input_patch * self.filter[n]))

        return output
    
    def getModel(self):
        filter_list = [filter_.tolist() for filter_ in self.filter]
        bias_list = self.bias.tolist()
        model = {
            "type": "conv2d",
            "params":{
                "kernel": filter_list ,
                "bias": bias_list
            }
        }

        return model

In [98]:
import numpy as np
class Flatten:
    def __init__(self):
        pass

    def forward(self, input_data):
        # Mendapatkan ukuran input
        input_shape = input_data.shape

        # Melakukan flatten, mengubah tensor tiga dimensi menjadi tensor dua dimensi
        output_data = input_data.flatten()

        return output_data
    
    def getModel(self):
        return ''

    

In [99]:
import numpy as np
class DenseLayer:
    def __init__(self, input_size, num_units, activation_function):
        self.input_size = input_size
        self.num_units = num_units
        self.activation_function = activation_function

        # Initialize weights and bias with random values
        self.weights = np.random.randn(input_size, num_units)
        self.bias = np.zeros(num_units)

    def forward(self, input_data):
        # Perform matrix multiplication (input_data * weights) and add bias
        pre_activation = np.dot(input_data, self.weights) + self.bias

        # Apply activation function based on the chosen mode
        if self.activation_function == "relu":
            output_data = self.relu(pre_activation)
        elif self.activation_function == "sigmoid":
            output_data = self.sigmoid(pre_activation)
        else:
            raise ValueError("Invalid activation function. Choose 'relu' or 'sigmoid'.")

        return output_data

    def relu(self, x):
        return np.maximum(0, x)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def getModel(self):
        weight_list = self.weights.tolist()
        bias_list = self.bias.tolist()
        model = {
            "type": "dense",
            "params":{
                "kernel": weight_list ,
                "bias": bias_list,
                "activation": self.activation_function
            }
        }

        return model
    

In [107]:
import json

class Model: 
    def __init__(self):
        self.layers =[]

    def train_network(self,train_data, label_data, batch_size, lr=0.01, epochs=200):
        batch_train = np.array_split(train_data,batch_size)
        batch_label = np.array_split(label_data, batch_size)
        for epoch in range(epochs):
            total_loss = 0.0
            correct_predictions = 0
            num = 0

            for i in range(len(batch_train)):
                for j in range(len(batch_train[i])):
                # for j in range(10):
                    input = batch_train[i][j]
                    for layer in self.layers:
                        output = layer.forward(input)
                        input = output
                    predictions = output[0]
                    if (predictions>0.5):
                        predict = 1
                    else :
                        predict = 0
                    num +=1
                    if predict == batch_label[i][j]:
                        correct_predictions += 1
            accuracy = (correct_predictions / num) * 100.0
            print(f"Epoch {epoch + 1}/{epochs} - Accuracy: {accuracy:.2f}%")    
                
        return 0
        
    def add(self,layer):
        self.layers.append(layer)

    def saveModel(self):
        save_model = []
        for layer in self.layers:
            out_model = layer.getModel()
            if out_model != '':
                save_model.append(out_model)
        file_name = 'model.json'

        # Open the file in write mode and write the save_model list to it as JSON
        with open(file_name, 'w') as json_file:
            json.dump(save_model, json_file)


In [109]:
samples = next(train_images)
input_shape = samples[0][0].shape
conv = Convolution(input_size=input_shape, padding_size=1, filter_size=(3, 3), num_filters=2, stride=1, bias=0)
pool = Pooling(filter_size = (32,32), stride=32, mode='max')
flat = Flatten()
dense_layer_relu = DenseLayer(input_size=128, num_units=7, activation_function="sigmoid")
dense_layer_sigmoid = DenseLayer(input_size=7, num_units=1, activation_function="relu")
model = Model()
model.add(conv )
model.add(pool)
model.add(flat)
model.add(dense_layer_relu)
model.add(dense_layer_sigmoid)


In [47]:
model.train_network(train_data=X_train, label_data=y_train, batch_size=1, lr=0.01, epochs=1)

Epoch 1/1 - Accuracy: 50.00%


0

In [110]:
model.saveModel()

In [82]:
a=dense_layer_relu.getModel()
b=dense_layer_sigmoid.getModel()
print(a)
print(b)

{'type': 'dense', 'params': {'kernel': [[-0.09901504883897505, 1.703824186900118, -0.2409769865441402, 0.6248644324682857, -0.43515433941654186, -1.0472295065090846, -0.4644540165078385], [-1.1645162105438027, 1.4850383812195946, 1.5927971797206957, -1.1783797165933647, -1.2913574508767867, 1.1018542651055536, -0.8721284001136266], [1.3039928635069722, 0.5499984326427169, -1.8457470546226256, -0.4879318637241982, 0.7969434510423677, -2.4895540337683624, -0.021883053194361922], [0.3091981602336915, 0.5226193752359098, 0.642232798778753, 0.3718399161631764, 1.2948345854019938, -0.5344442141720189, 1.2084343724918223], [-0.1655672068024553, -0.5609093685843438, 1.2059319754795135, -0.48904048776844977, -0.26197922659019823, -1.7016213865699947, -0.05210604316126182], [1.529423945886405, -0.5162884573304174, 1.0199714422611028, 0.3948831217467353, -0.07794510841011958, -0.5732489903291376, -0.4073215954287192], [1.4676152986762667, 0.03540284971889111, -0.595568209419227, -0.27842027852939

In [ ]:
# samples = next(train_images)
# input_shape = samples[0][0].shape
# conv = Convolution(input_size=input_shape, padding_size=1, filter_size=(3, 3), num_filters=2, stride=1, biases=0)
# pool = Pooling(filter_size = (32,32), stride=32, mode='max')
# flat = Flatten()
# dense_layer_relu = DenseLayer(input_size=128, num_units=7, activation_function="sigmoid")
# dense_layer_sigmoid = DenseLayer(input_size=7, num_units=1, activation_function="relu")

In [40]:

pool_out =''
def train_network(train_data , conv, pool, lr=0.01, epochs=200):
    for epoch in range(epochs):
        total_loss = 0.0
        correct_predictions = 0
        num = 0

        # for i in range(1):
        # for i in range(len(train_data)):
        for i in range(1):
            for j in range(10):
                # # Forward pass    
                conv_out = conv.forward(X_train[j])
                pool_out = pool.forward(conv_out)
                flatten_output = flat.forward(pool_out)
                dense_relu_ouput = dense_layer_relu.forward(flatten_output)
                dense_sigmoid_output = dense_layer_sigmoid.forward(dense_relu_ouput)
                predictions = dense_sigmoid_output[0]
                print(predictions)
                if (predictions>0.5):
                    predict = 1
                else :
                    predict = 0
                # Hitung prediksi
                if predict == y_train[j]:
                    correct_predictions += 1
                num +=1
        print(f"Batch {i}")
        accuracy = (correct_predictions / num) * 100.0
        print(f"Epoch {epoch + 1}/{epochs} - Accuracy: {accuracy:.2f}%")
    return accuracy

output =  train_network(train_images, conv, pool, lr=0.01, epochs=1)




0.1888545188360274
0.14586901157902146
0.16289652505998556
0.14553522744365613
0.23745234647551713
0.14818549144885382
0.0
0.0
0.15730099205619522
0.1451121955765984
Batch 0
Epoch 1/1 - Accuracy: 70.00%


In [ ]:
print(len(train_images[0][0]))

In [ ]:
import tensorflow as tf
import numpy as np
input_data = np.array(    [[[1, 2, 3, 4],
                           [5, 6, 7, 8],
                           [9, 10, 11, 12]],
                          [[13, 14, 15, 16],
                           [17, 18, 19, 20],
                           [21, 22, 23, 24]]])
# input_data.flatten()

# # Create a Flatten layer
flatten_layer = tf.keras.layers.Flatten()

# # Apply the Flatten layer to the input tensor
output_data = flatten_layer(input_data)

# # Print the flattened output
print(output_data)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, 3, 3, input_shape=(3, 32, 32)))
model.output_shape
model.add(tf.keras.layers.Flatten())
model.output_shape